# Problem Statement:

Goal:
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric:
1. Submissions are evaluated on Mean-Absolute-Error (MAE).
2. Submission File Format
The file should contain a header and have only two colums : Id and predicted SalePrice

## Import Dataset

In [1]:
import pandas as pd
A=pd.read_csv('/Users/sourabhdabke/Documents/ETL_Hive/Python/Week_14_Proj2/training_set.csv')

### Configuring correct display settings

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_rows',81)
pd.set_option('display.max_columns',811)

## Missing Data Treatment
### Here all the missing string values are substituted with their corresponding given states in the problem statement. For eg. 'na' in Alley is not actually a value not present but rather 'No Alley access'

In [3]:
A[['Alley']]=A[['Alley']].fillna('No alley access')
A[['BsmtQual','BsmtCond','BsmtExposure',
   'BsmtFinType1','BsmtFinType2']]=A[['BsmtQual','BsmtCond',
                                      'BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('No Basement')
A[['FireplaceQu']]=A[['FireplaceQu']].fillna('No Fireplace')
A[['GarageType','GarageYrBlt',
   'GarageFinish','GarageQual',
   'GarageCond']]=A[['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']].fillna('No Garage')
A[['PoolQC']]=A[['PoolQC']].fillna('No Pool')
A[['Fence']]=A[['Fence']].fillna('No Fence')
A[['MiscFeature']]=A[['MiscFeature']].fillna('None')

In [4]:
for i in A.columns:
    if A[i].dtype=='object':
        x=A[i].mode()[0]
        A[i]=A[i].fillna(x)
    else:
        x=A[i].mean()
        A[i]=A[i].fillna(x)

## Getting data in standard format by eliminating skewness

In [5]:
Q = A.skew()
col_with_skew = list(Q[Q>2].index)

In [6]:
import numpy as np
from numpy import log

for j in col_with_skew:
    W = []
    for i in A[j]:
        if (i != 0):
            W.append(log(i))
        else:
            W.append(0)
    A[j] = W

In [7]:
cat = []
con = []

for i in A.columns:
    if A[i].dtypes == 'object':
        cat.append(i)
    else:
        con.append(i)

## Exploratory Data Analysis

### Defining ANOVA to find important categorical columns

In [8]:
def ANOVA(df,cat,con):
    from statsmodels.formula.api import ols
    relation = con + " ~ " + cat
    model = ols(relation,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    return round(anova_results.iloc[0,4],4)

### Using ANOVA to calculate important categorical columns

In [9]:
from batch6pm import ANOVA

imp_cat_col = []
Q = list(cat)
for i in Q:
    print('-----------')
    print('SalePrice v/s',i)
    W = ANOVA(A,i,'SalePrice')
    print(W)
    if(W<0.05):
        imp_cat_col.append(i)


-----------
SalePrice v/s MSZoning
0.0
-----------
SalePrice v/s Street
0.117
-----------
SalePrice v/s Alley
0.0
-----------
SalePrice v/s LotShape
0.0
-----------
SalePrice v/s LandContour
0.0
-----------
SalePrice v/s Utilities
0.5847
-----------
SalePrice v/s LotConfig
0.0
-----------
SalePrice v/s LandSlope
0.1414
-----------
SalePrice v/s Neighborhood
0.0
-----------
SalePrice v/s Condition1
0.0
-----------
SalePrice v/s Condition2
0.0434
-----------
SalePrice v/s BldgType
0.0
-----------
SalePrice v/s HouseStyle
0.0
-----------
SalePrice v/s RoofStyle
0.0
-----------
SalePrice v/s RoofMatl
0.0
-----------
SalePrice v/s Exterior1st
0.0
-----------
SalePrice v/s Exterior2nd
0.0
-----------
SalePrice v/s MasVnrType
0.0
-----------
SalePrice v/s ExterQual
0.0
-----------
SalePrice v/s ExterCond
0.0
-----------
SalePrice v/s Foundation
0.0
-----------
SalePrice v/s BsmtQual
0.0
-----------
SalePrice v/s BsmtCond
0.0
-----------
SalePrice v/s BsmtExposure
0.0
-----------
SalePrice v/s

## Using correlation to calculate important continuous columns

In [10]:
q = A[con].corr()['SalePrice']
imp_con_col = q[(q<1)&(q>0.4)].index
imp_con_col

Index(['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageCars', 'GarageArea', 'OpenPorchSF'],
      dtype='object')

In [11]:
Anew = A[imp_con_col].join(A[imp_cat_col])

## Preprocessing and getting data ready for back propogation

In [12]:
cat = []
con = []

for i in Anew.columns:
    if Anew[i].dtypes == 'object':
        cat.append(i)
    else:
        con.append(i)

In [13]:
X1 = pd.get_dummies(Anew[cat])

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X2 = pd.DataFrame(ss.fit_transform(Anew[con]),columns=con)

Xnew = X2.join(X1)


## Define Y

In [14]:
Y = A[['SalePrice']]
Y.shape

(1460, 1)

## Create Backward Elimination Model

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=21)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)

ols = OLS(ytrain,xconst)

model = ols.fit()
model.rsquared_adj

0.9182619677985082

In [16]:
p = model.pvalues
q = model.pvalues.max()

col_to_drop = list(p[p == q].index)[0]
col_to_drop

'RoofStyle_Hip'

### First model with 'RoofStyle_Hip' removed

In [17]:
p = model.pvalues
q = model.pvalues.max()

col_to_drop = list(p[p==q].index)[0]
print(col_to_drop)
Xnew = Xnew.drop(labels=col_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=21)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)

ols = OLS(ytrain,xconst)

model = ols.fit()
model.rsquared_adj


RoofStyle_Hip


0.9182619677985082

### Created successive models removing more irrelevant columns till 'rsquared_adj' is maximum

In [211]:
p = model.pvalues
q = model.pvalues.max()

col_to_drop = list(p[p==q].index)[0]
print(col_to_drop)
Xnew = Xnew.drop(labels=col_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=21)

from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)

ols = OLS(ytrain,xconst)

model = ols.fit()
model.rsquared_adj


GarageYrBlt_1984.0


0.9280063724965251

### So the best model with maximum explained variance is 92.8 % with 175 columns 

## Creating Prediction Based on testing data

#### Reading testing dataset

In [328]:
B = pd.read_csv('/Users/sourabhdabke/Documents/ETL_Hive/Python/Week_14_Proj2/testing_set.csv')

#### Missing data treatment

In [329]:
B[['Alley']]=A[['Alley']].fillna('No alley access')
B[['BsmtQual','BsmtCond','BsmtExposure',
   'BsmtFinType1','BsmtFinType2']]=B[['BsmtQual','BsmtCond',
                                      'BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('No Basement')
B[['FireplaceQu']]=B[['FireplaceQu']].fillna('No Fireplace')
B[['GarageType','GarageYrBlt',
   'GarageFinish','GarageQual',
   'GarageCond']]=B[['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']].fillna('No Garage')
B[['PoolQC']]=B[['PoolQC']].fillna('No Pool')
B[['Fence']]=B[['Fence']].fillna('No Fence')
B[['MiscFeature']]=B[['MiscFeature']].fillna('None')

In [330]:
B1 = B.drop(labels=['Id'],axis = 1)

for i in B1.columns:
    if B1[i].dtype=='object':
        x=B1[i].mode()[0]
        B1[i]=B1[i].fillna(x)
    else:
        x=B1[i].mean()
        B1[i]=B1[i].fillna(x)

cat = []
con = []

for i in B1.columns:
    if B1[i].dtypes == 'object':
        cat.append(i)
    else:
        con.append(i)
X1 = pd.get_dummies(B1[cat])

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X2 = pd.DataFrame(ss.fit_transform(B1[con]),columns=con)

tsd = X2.join(X1)

### Choosing test data colums from the same number of columns we got from backward elimination model. If columns are missing then adding them and setting their values to zero

In [331]:
Q = list(Xnew.columns)

In [332]:
#tsd[Q]

In [333]:
for i in ['Condition2_RRAe', 'Condition2_RRNn', 'HouseStyle_2.5Fin',
          'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal',
          'RoofMatl_Roll', 'Heating_OthW', 'GarageYrBlt_1908.0',
          'GarageYrBlt_1914.0', 'GarageQual_Ex', 'PoolQC_Fa', 'MiscFeature_TenC']:
    tsd[i]=0

In [334]:
final = tsd[Q]

### Testing data has one less row, so adding a null row for prediction

In [335]:
q = []
for i in range(final.shape[1]):
    q.append(0)
q = pd.DataFrame(q).T
q.index = range(final.shape[0],(final.shape[0]+1))
q.columns = final.columns
final = final.append(q)

In [336]:
final.shape

(1460, 175)

In [337]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(final,Y,test_size=.2,random_state=21)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

model = lr.fit(final,Y)
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

from sklearn.metrics import mean_absolute_error
bias = mean_absolute_error(ytrain,tr_pred)
var = mean_absolute_error(ytest,ts_pred)
list([bias,var])


[54800.19106473295, 56021.951023122005]

## Using Ridge and Lasso regression techniques to further reduce the error

### Tuning Grid

In [338]:
val_to_try = []
x=1
for i in range(0,50):
    x = x + 0.001
    val_to_try.append(round(x,3))
x=1
for i in range(0,50):
    x = x - 0.001
    val_to_try.append(round(x,3))
    
tuning_grid = {'alpha':val_to_try}

# Ridge

In [339]:
from sklearn.linear_model import Ridge
rr = Ridge()

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rr,tuning_grid,scoring='neg_mean_absolute_error',cv = 4)

cvmodel = cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 1.05}

# Lasso

In [340]:
from sklearn.linear_model import Lasso
lr = Lasso()

cv = GridSearchCV(lr,tuning_grid,scoring='neg_mean_absolute_error',cv=4)

cvmodel = cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 1.05}

# Lasso Model

In [348]:
lr = Lasso(alpha=1.05)

model = lr.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error
bias = mean_absolute_error(ytrain,pred_tr)
var = mean_absolute_error(ytest,pred_ts)

list([bias,var])

[53911.26733018525, 63610.805455874885]

# Ridge Model

In [343]:
rr = Ridge(alpha=1.05)

model = rr.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error
bias = mean_absolute_error(ytrain,pred_tr)
var = mean_absolute_error(ytest,pred_ts)

list([bias,var])

[54190.729326854074, 62553.71107936068]

# Best is Lasso Model so created predictions based on the same

In [344]:
T = B[['Id']]

In [356]:
pred = model.predict(final)
pred = pred[0:1459]

In [357]:
T['Predicted'] = pred

In [358]:
T

,Id,Predicted
0,1461,199509.185272
1,1462,166516.322965
2,1463,187618.638729
3,1464,173607.816714
4,1465,225603.053280
...,...,...
1454,2915,170532.294209
1455,2916,185063.290709
1456,2917,200980.450370
1457,2918,142431.011891


In [359]:
T.to_csv('/Users/sourabhdabke/Desktop/Housing_submission.csv',index=None)